In [1]:
import pyspark
from pyspark.sql import SparkSession, Row

from datetime import datetime,date

In [14]:
#https://iceberg.apache.org/spark-quickstart/#adding-a-catalog
conf = (
    pyspark.SparkConf()
        .setAppName('test')
  		#packages
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.3,software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178')
  		#SQL Extensions
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')
  		#Configuring Catalog
        .set('spark.sql.catalog.local', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.defaultCatalog','local')
        .set('spark.sql.catalog.local.type', 'hadoop')
        .set('spark.sql.catalog.local.warehouse', 'warehouse')
)

spark = SparkSession.builder.config(conf=conf).getOrCreate()
print(f"Spark Running")
spark

Spark Running


In [5]:
df_emp = spark.createDataFrame([
    Row(id=1, age=21, country='USA', birth_date=date(2000, 8, 1)),
   
    Row(id=2, age=21, country='IN', birth_date=date(2000, 6, 2)),
   
    Row(id=4, age=21, country='USA', birth_date=date(2000, 5, 3))
])
 

In [6]:
df_emp.show()

+---+---+-------+----------+
| id|age|country|birth_date|
+---+---+-------+----------+
|  1| 21|    USA|2000-08-01|
|  2| 21|     IN|2000-06-02|
|  4| 21|    USA|2000-05-03|
+---+---+-------+----------+



In [8]:
df_emp.repartition("country").write.mode("overwrite").parquet("sample_write_dataset/non_partitioned/emp_parquet/") 
df_emp.repartition("country").write.mode("overwrite").partitionBy("country").parquet("sample_write_dataset/partitioned/emp_parquet/")

In [34]:
!ls sample_write_dataset/non_partitioned/emp_parquet/ -a

.
..
part-00000-e6991d84-d761-4963-87d1-ddf48c2b4fe6-c000.snappy.parquet
.part-00000-e6991d84-d761-4963-87d1-ddf48c2b4fe6-c000.snappy.parquet.crc
_SUCCESS
._SUCCESS.crc


In [15]:
df_emp.writeTo("test.employees").create()

In [18]:
spark.sql("select * from test.employees").collect()

[Row(id=1, age=21, country='USA', birth_date=datetime.date(2000, 8, 1)),
 Row(id=2, age=21, country='IN', birth_date=datetime.date(2000, 6, 2)),
 Row(id=4, age=21, country='USA', birth_date=datetime.date(2000, 5, 3))]

In [16]:
#df_emp.write.format("delta").save("sampel_write_dataset/partitioned/emp_delta/")